In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
os.chdir('drive/My Drive/DataDays/second_round')

In [0]:
df = pd.read_csv('divar_posts_dataset.csv')

In [0]:
df = df.drop(columns=['Unnamed: 0', 'archive_by_user', 'brand', 
       'city', 'created_at',  'id', 'image_count', 'mileage',
       'platform', 'price', 'type', 'year'])

In [0]:
df['cat1_2'] = df.cat1.astype(str).str.cat(df.cat2.astype(str), sep='|')
df['cat1_2_3'] = df.cat1_2.astype(str).str.cat(df.cat3.astype(str), sep='|')
df['text']=df.title.astype(str).str.cat(df.desc.astype(str),sep=' ')

In [0]:
df = df.drop(columns=['cat1', 'cat2', 'cat3', 'desc', 'title', 'cat1_2'])

In [0]:
labels = {j:i for i,j in enumerate(df.cat1_2_3.unique())}


In [0]:
df['cat1_2_3']=df['cat1_2_3'].apply(lambda i : labels[i])

In [0]:
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [0]:
df_trn, df_val = train_test_split(df, stratify = df['cat1_2_3'], test_size = 0.2, random_state = 12)


In [0]:
stop_words=["و","در","به","از","که","این","را","با","است","برای","آن","یک","خود","تا","کرد","بر","هم","نیز","گفت","می‌شود","وی","شد","دارد","ما","اما","یا","شده","باید","هر","آنها","بود","او","دیگر","دو","مورد","می‌کند","شود","کند","وجود","بین","پیش","شده_است","پس","نظر","اگر","همه","یکی","حال","هستند","من","کنند","نیست","باشد","چه","می","بخش","می‌کنند","همین","افزود","هایی","دارند","راه","همچنین","روی","داد","بیشتر","بسیار","سه","داشت","چند","سوی","تنها","هیچ","میان","اینکه","شدن","بعد","جدید","ولی","حتی","کردن","برخی","کردند","می‌دهد","اول","نه","کرده_است","نسبت","بیش","شما","چنین","طور","افراد","تمام","درباره","بار","بسیاری","می‌تواند","کرده","چون","ندارد","دوم","بزرگ","طی","حدود","همان","بدون","البته","آنان","می‌گوید","دیگری","خواهد_شد","کنیم","قابل","یعنی","رشد","می‌توان","وارد","کل","ویژه","قبل","براساس","نیاز","گذاری","هنوز","لازم","سازی","بوده_است","چرا","می‌شوند","وقتی","گرفت","کم","جای","حالی","تغییر","پیدا","اکنون","تحت","باعث","مدت","فقط","زیادی","تعداد","آیا","بیان","رو","شدند","عدم","کرده_اند","بودن","نوع","بلکه","جاری","دهد","برابر","مهم","بوده","اخیر","مربوط","امر","زیر","گیری","شاید","خصوص","آقای","اثر","کننده","بودند","فکر","کنار","اولین","سوم","سایر","کنید","ضمن","مانند","باز","می‌گیرد","ممکن","حل","دارای","پی","مثل","می‌رسد","اجرا","دور","منظور","کسی","موجب","طول","امکان","آنچه","تعیین","گفته","شوند","جمع","خیلی","علاوه","گونه","تاکنون","رسید","ساله","گرفته","شده_اند","علت","چهار","داشته_باشد","خواهد_بود","طرف","تهیه","تبدیل","مناسب","زیرا","مشخص","می‌توانند","نزدیک","جریان","روند","بنابراین","می‌دهند","یافت","نخستین","بالا","پنج","ریزی","عالی","چیزی","نخست","بیشتری","ترتیب","شده_بود","خاص","خوبی","خوب","شروع","فرد","کامل","غیر","می‌رود","دهند","آخرین","دادن","جدی","بهترین","شامل","گیرد","بخشی","باشند","تمامی","بهتر","داده_است","حد","نبود","کسانی","می‌کرد","داریم","علیه","می‌باشد","دانست","ناشی","داشتند","دهه","می‌شد","ایشان","آنجا","گرفته_است","دچار","می‌آید","لحاظ","آنکه","داده","بعضی","هستیم","اند","برداری","نباید","می‌کنیم","نشست","سهم","همیشه","آمد","اش","وگو","می‌کنم","حداقل","طبق","جا","خواهد_کرد","نوعی","چگونه","رفت","هنگام","فوق","روش","ندارند","سعی","بندی","شمار","کلی","کافی","مواجه","همچنان","زیاد","سمت","کوچک","داشته_است","چیز","پشت","آورد","حالا","روبه","سال‌های","دادند","می‌کردند","عهده","نیمه","جایی","دیگران","سی","بروز","یکدیگر","آمده_است","جز","کنم","سپس","کنندگان","خودش","همواره","یافته","شان","صرف","نمی‌شود","رسیدن","چهارم","یابد","متر","ساز","داشته","کرده_بود","باره","نحوه","کردم","تو","شخصی","داشته_باشند","محسوب","پخش","کمی","متفاوت","سراسر","کاملا","داشتن","نظیر","آمده","گروهی","فردی","ع","همچون","خطر","خویش","کدام","دسته","سبب","عین","آوری","متاسفانه","بیرون","دار","ابتدا","شش","افرادی","می‌گویند","سالهای","درون","نیستند","یافته_است","پر","خاطرنشان","گاه","جمعی","اغلب","دوباره","می‌یابد","لذا","زاده","گردد","اینجا"]

In [0]:
tokenized_doc = df['text'].apply(lambda x: x.split())

# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# de-tokenization 
detokenized_doc = [] 
for i in range(len(df)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

df['text'] = detokenized_doc

In [0]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=256)

In [0]:
learn = language_model_learner(data_lm,arch=AWD_LSTM, drop_mult=0.7)


In [0]:
learn.fit_one_cycle(1, 1e-2)


epoch,train_loss,valid_loss,accuracy,time
1,4.971684,4.639180,0.233429,51:52


In [0]:
learn.save_encoder('ft_enc')


In [0]:
learn = text_classifier_learner(data_clas,arch=AWD_LSTM, drop_mult=0.7)
learn.load_encoder('ft_enc')

In [0]:
learn.fit_one_cycle(5, 1e-2)


epoch,train_loss,valid_loss,accuracy,time
1,2.449938,1.079201,0.724957,13:39
2,2.426211,1.071944,0.724525,13:23
3,2.338404,0.964258,0.746400,12:57
4,2.330456,0.975453,0.743847,13:52
5,2.357829,0.952913,0.748991,13:49


In [0]:
learn.save("trained_model", return_path=True)
learn.load('tra')

PosixPath('models/trained_model.pth')